<a href="https://colab.research.google.com/github/tomknightatl/USCCB/blob/main/Build_Parishes_Database_Using_AgenticAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Cell 1:
# Import required code
!wget https://raw.githubusercontent.com/tomknightatl/USCCB/main/llm_utils.py

--2025-05-27 20:02:10--  https://raw.githubusercontent.com/tomknightatl/USCCB/main/llm_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1638 (1.6K) [text/plain]
Saving to: ‘llm_utils.py’

llm_utils.py        100%[===================>]   1.60K  --.-KB/s    in 0s      

2025-05-27 20:02:11 (28.6 MB/s) - ‘llm_utils.py’ saved [1638/1638]



In [1]:
# Cell 1: Import required libraries
!pip install supabase google-generativeai psycopg2-binary tenacity

import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import os
from google.colab import userdata
from openai import OpenAI
from urllib.parse import urlparse
import json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.15
    Uninstalling aiohttp-3.11.15:
      Successfully uninstalled aiohttp-3.11.15


In [2]:
# Cell 3: User-configurable parameters, Supabase Setup, and Data Retrieval

from google.colab import userdata
import google.generativeai as genai
from supabase import create_client, Client
import os # For environment variables if needed, and MAX_URLS_TO_PROCESS logic
import random

print("--- User Configurable Parameters & Supabase Setup ---")

# --- Processing Limit Configuration ---
# Set the maximum number of parish directory URLs to process (None = process all)
MAX_URLS_TO_PROCESS = 5  # Change this number or set to None to process all
                         # STARTING WITH A SMALL NUMBER FOR TESTING

if MAX_URLS_TO_PROCESS:
    print(f"Processing will be limited to {MAX_URLS_TO_PROCESS} randomly selected URLs.")
else:
    print("Processing will include all relevant URLs from DiocesesParishDirectory.")

# --- Supabase Configuration ---
SUPABASE_URL = None
SUPABASE_KEY = None
SUPABASE_URL_FROM_USERDATA = userdata.get('SUPABASE_URL')
SUPABASE_KEY_FROM_USERDATA = userdata.get('SUPABASE_KEY')

if SUPABASE_URL_FROM_USERDATA:
    SUPABASE_URL = SUPABASE_URL_FROM_USERDATA
if SUPABASE_KEY_FROM_USERDATA:
    SUPABASE_KEY = SUPABASE_KEY_FROM_USERDATA

supabase: Client = None
if SUPABASE_URL and SUPABASE_KEY:
    try:
        supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
        print("Supabase client initialized successfully.")
    except Exception as e:
        print(f"Error initializing Supabase client: {e}")
        supabase = None
else:
    print("Supabase URL and/or Key NOT loaded. Please check Colab Secrets.")
    print("Required secrets: SUPABASE_URL, SUPABASE_KEY")

# --- GenAI API Key Setup ---
GENAI_API_KEY_FROM_USERDATA = userdata.get('GENAI_API_KEY_USCCB') # Assuming same secret name as other notebook
GENAI_API_KEY = None

if GENAI_API_KEY_FROM_USERDATA and GENAI_API_KEY_FROM_USERDATA not in ["YOUR_API_KEY_PLACEHOLDER", "SET_YOUR_KEY_HERE"]:
    GENAI_API_KEY = GENAI_API_KEY_FROM_USERDATA

if GENAI_API_KEY:
    try:
        genai.configure(api_key=GENAI_API_KEY)
        print("GenAI configured successfully.")
    except Exception as e:
        print(f"Error configuring GenAI with key: {e}. GenAI features might not work.")
        GENAI_API_KEY = None # Ensure it's None if configuration fails
else:
    print("GenAI API Key is not set (Secret: GENAI_API_KEY_USCCB). LLM features will not work.")

# --- Data Retrieval from Supabase ---
urls_to_process = []
if supabase:
    try:
        print("Fetching parish directory URLs from DiocesesParishDirectory table...")
        # Fetch non-null, non-empty parish_directory_url
        query = supabase.table('DiocesesParishDirectory').select('parish_directory_url').not_.is_('parish_directory_url', 'null').not_.eq('parish_directory_url', '')

        response = query.execute()

        if response.data:
            fetched_urls = [item['parish_directory_url'] for item in response.data if item['parish_directory_url']]
            print(f"Successfully fetched {len(fetched_urls)} URLs from Supabase.")

            if MAX_URLS_TO_PROCESS and len(fetched_urls) > MAX_URLS_TO_PROCESS:
                urls_to_process = random.sample(fetched_urls, MAX_URLS_TO_PROCESS)
                print(f"Randomly selected {len(urls_to_process)} URLs for processing.")
            else:
                urls_to_process = fetched_urls
                print(f"Processing all {len(urls_to_process)} fetched URLs.")
        else:
            print("No parish directory URLs found in DiocesesParishDirectory or error in fetching.")
            if hasattr(response, 'error') and response.error:
                 print(f"Supabase error: {response.error}")


    except Exception as e:
        print(f"Error fetching URLs from Supabase: {e}")
        urls_to_process = []
else:
    print("Supabase client not initialized. Cannot fetch URLs.")

if not urls_to_process:
    print("No URLs to process. Further steps might be skipped or fail.")
else:
    print(f"Prepared {len(urls_to_process)} URLs for processing.")

# For subsequent cells to use, we will name the list of URLs `urls` as in the original notebook
urls = [(url,) for url in urls_to_process] # Keep the tuple structure if downstream code expects it

print("--- End User Configurable Parameters & Supabase Setup ---")

--- User Configurable Parameters & Supabase Setup ---
Processing will be limited to 5 randomly selected URLs.
Supabase client initialized successfully.
GenAI configured successfully.
Fetching parish directory URLs from DiocesesParishDirectory table...
Successfully fetched 192 URLs from Supabase.
Randomly selected 5 URLs for processing.
Prepared 5 URLs for processing.
--- End User Configurable Parameters & Supabase Setup ---


In [5]:
# Cell 4: Process each URL using Gemini API
from llm_utils import invoke_gemini_model
import json
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

def extract_domain(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

def process_url_with_gemini(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract visible text from the webpage
    visible_text = ' '.join([s for s in soup.stripped_strings])

    # Prepare the prompt for Gemini
    prompt = f"""
    Analyze the following webpage content, which is from the URL {url}.
    Extract parish information. The information should include:
    Name, Status, Deanery, EST (Established Date), Street Address, City, State, Zipcode, Phone Number, and Website.
    If any specific piece of information is not found or not applicable, use the JSON value null for that field.
    Format the output as a single, valid JSON object with these exact keys:
    "Name", "Status", "Deanery", "EST", "Street Address", "City", "State", "Zipcode", "Phone Number", "Website".

    Webpage content (first 40000 characters):
    {visible_text[:40000]}
    """

    # Call Gemini API
    try:
        # Ensure GENAI_API_KEY is loaded and genai is configured (done in Cell 3)
        if 'GENAI_API_KEY' not in globals() or not GENAI_API_KEY:
            print("GenAI API Key not configured. Skipping LLM call.")
            return None

        # Call the shared Gemini function
        response_text = invoke_gemini_model(prompt_text=prompt) # invoke_gemini_model is from llm_utils

        print(f"Gemini API Response: {response_text}") # Log the raw API response

        # Attempt to parse the JSON response
        # Gemini might sometimes wrap JSON in ```json ... ```, so try to strip that
        if response_text.strip().startswith("```json"):
            content_to_parse = response_text.strip()[7:-3].strip()
        elif response_text.strip().startswith("```"): # Generic backticks
            content_to_parse = response_text.strip()[3:-3].strip()
        else:
            content_to_parse = response_text.strip()

        extracted_data = json.loads(content_to_parse)
        return extracted_data
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {str(e)}")
        print(f"Raw API Response that failed parsing: {response_text}")
        # Return a dict with nulls to indicate parsing failure but allow DB storage of this failure
        return {"Name": None, "Status": "JSON Decode Error", "Deanery": None, "EST": None,
                "Street Address": str(e), "City": None, "State": None, "Zipcode": None,
                "Phone Number": None, "Website": None, "source_url": url, "domain": extract_domain(url)}
    except Exception as e:
        print(f"Error calling Gemini API: {str(e)}")
        # Return a dict with nulls to indicate API error
        return {"Name": None, "Status": "API Error", "Deanery": None, "EST": None,
                "Street Address": str(e), "City": None, "State": None, "Zipcode": None,
                "Phone Number": None, "Website": None, "source_url": url, "domain": extract_domain(url)}

# Process each URL
for url_tuple in urls: # Assuming urls is a list of tuples from Cell 3
    url = url_tuple[0]
    print(f"Processing URL: {url}")

    try:
        parish_data = process_url_with_gemini(url)

        if parish_data and supabase: # Ensure data and supabase client exist
            # Add source_url and domain if not already added by error handling in process_url_with_gemini
            if 'source_url' not in parish_data:
                 parish_data['source_url'] = url
            if 'domain' not in parish_data:
                 parish_data['domain'] = extract_domain(url)

            # Prepare data for Supabase, ensuring all keys are present, defaulting to None if missing
            data_to_upsert = {
                'Name': parish_data.get('Name'),
                'Status': parish_data.get('Status'),
                'Deanery': parish_data.get('Deanery'),
                'EST': parish_data.get('EST'),
                # Supabase table uses 'StreetAddress', JSON uses 'Street Address'
                'StreetAddress': parish_data.get('Street Address'),
                'City': parish_data.get('City'),
                'State': parish_data.get('State'),
                'Zipcode': parish_data.get('Zipcode'),
                'PhoneNumber': parish_data.get('Phone Number'), # Supabase table uses 'PhoneNumber'
                'Website': parish_data.get('Website'),
                'source_url': parish_data['source_url'],
                'domain': parish_data['domain']
            }

            try:
                # Upsert into Parishes table. Assuming 'source_url' can be a unique identifier for upsert.
                # If not, adjust conflict resolution or use insert.
                # For now, using 'source_url' as the conflict target for an upsert.
                # This implies 'source_url' column in 'Parishes' table must have a unique constraint.
                response = supabase.table('Parishes').upsert(data_to_upsert, on_conflict='source_url').execute()

                if hasattr(response, 'error') and response.error:
                    print(f"Error upserting data to Supabase for {url}: {response.error}")
                else:
                    print(f"Data upserted to Supabase for: {data_to_upsert.get('Name', 'Unknown Parish')} from {url}")

            except Exception as supa_error:
                print(f"Supabase API error during upsert for {url}: {supa_error}")
        elif not supabase:
            print(f"Supabase client not available. Skipping database write for {url}.")
        elif not parish_data:
             print(f"No data extracted for {url}. Skipping database write.")

    except Exception as e:
        print(f"Error processing {url}: {str(e)}")

print("All URLs processed.")

Processing URL: https://www.diopueblo.org/parishes
Gemini API Response: ```json
{
  "Name": null,
  "Status": null,
  "Deanery": null,
  "EST": null,
  "Street Address": null,
  "City": null,
  "State": null,
  "Zipcode": null,
  "Phone Number": null,
  "Website": null
}
```

Supabase API error during upsert for https://www.diopueblo.org/parishes: {'message': 'JSON could not be generated', 'code': 404, 'hint': 'Refer to full message for details', 'details': "b'{}'"}
Processing URL: https://www.rcbo.org/directories/parishes/
Gemini API Response: ```json
{
  "Name": null,
  "Status": null,
  "Deanery": null,
  "EST": null,
  "Street Address": null,
  "City": null,
  "State": null,
  "Zipcode": null,
  "Phone Number": null,
  "Website": null
}
```

Supabase API error during upsert for https://www.rcbo.org/directories/parishes/: {'message': 'JSON could not be generated', 'code': 404, 'hint': 'Refer to full message for details', 'details': "b'{}'"}
Processing URL: https://archdiosf.org/dire